In [1]:
data_var = '2024-01-29'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10725,2024-01-29,Eua Nba,21:00,Charlotte Hornets,New York Knicks,3.22,1.38,223.5,1.94,2.00,5.5,2.06,1.69,nT8xzXzB,0.310559,0.724638,0.515464,0.500000,0.035197,0.6,0.2,NaN,NaN,598.052,529.246519,0.884951,185.220,50.908538,0.274854,353.588,232.498,528.26,172.80,1.0,0.0,0.0,0.0,0.565685,0.021536,0.139536,-3.44,-0.688,-3.226744,0.333954,0.2,-0.133954,2.46,0.492,0.772358,0.502093,0.6,0.097907
10726,2024-01-29,Eua Nba,21:00,Cleveland Cavaliers,Los Angeles Clippers,2.11,1.78,223.5,1.80,2.09,1.5,2.02,1.90,UibsZjjH,0.473934,0.561798,0.555556,0.478469,0.035731,0.2,0.2,NaN,NaN,318.776,78.990070,0.247792,212.476,97.280634,0.457843,196.668,223.674,342.90,385.25,1.0,0.0,0.0,0.0,0.119972,0.105430,0.043292,3.24,0.648,1.712963,0.625642,0.8,0.174358,1.65,0.330,2.363636,0.546554,0.6,0.053446
10727,2024-01-29,Eua Nba,21:30,Boston Celtics,New Orleans Pelicans,1.29,3.82,231.5,1.89,2.03,-8.5,2.02,1.92,8rcoYA5N,0.775194,0.261780,0.529101,0.492611,0.036974,0.4,0.6,NaN,NaN,164.816,32.353802,0.196303,185.126,34.008284,0.183703,147.942,293.502,134.40,220.80,0.0,0.0,0.0,0.0,0.700188,0.050508,0.035894,-1.48,-0.296,-0.979730,0.792840,0.8,0.007160,2.38,0.476,5.924370,0.544762,0.7,0.155238
10728,2024-01-29,Eua Nba,21:30,Brooklyn Nets,Utah Jazz,1.93,1.93,232.5,1.89,2.03,-1.5,2.07,1.84,G8dkXUKT,0.518135,0.518135,0.529101,0.492611,0.036269,0.2,0.6,NaN,NaN,315.260,131.749635,0.417908,267.696,103.252455,0.385708,202.042,358.140,165.36,171.52,0.0,0.0,0.0,0.0,0.000000,0.050508,0.083189,-3.44,-0.688,-1.351744,0.518015,0.3,-0.218015,4.03,0.806,1.153846,0.423349,0.6,0.176651
10729,2024-01-29,Eua Nba,21:30,Miami Heat,Phoenix Suns,1.62,2.39,228.5,1.83,1.93,-5.5,2.06,1.71,IHrHSlTp,0.617284,0.418410,0.546448,0.518135,0.035694,0.2,0.4,NaN,NaN,227.078,118.055540,0.519890,198.054,45.515404,0.229813,193.908,207.836,390.50,173.46,1.0,0.0,0.0,0.0,0.271557,0.037612,0.131293,-1.95,-0.390,-1.589744,0.589829,0.4,-0.189829,0.33,0.066,21.060606,0.557779,0.4,-0.157779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10786,2024-01-29,Turquia Tbl,12:00,Kocaeli BŞB Kağıtspor,Sigortam.net ITU,2.76,1.41,160.5,1.86,1.80,4.5,2.00,1.72,CzdogRRl,0.362319,0.709220,0.537634,0.555556,0.071539,0.0,0.0,NaN,NaN,233.580,100.289466,0.429358,127.586,17.826112,0.139718,0.000,0.000,335.82,141.51,1.0,0.0,0.0,0.0,0.457839,0.023184,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10787,2024-01-29,Turquia Tb2L,07:00,Aykanat Spor,Uskudar,1.76,1.97,141.5,1.85,1.85,-2.5,2.03,1.70,bwqkhH6E,0.568182,0.507614,0.540541,0.540541,0.075796,0.0,0.0,NaN,NaN,150.776,45.075789,0.298959,126.448,31.956939,0.252728,0.000,0.000,138.45,96.90,0.0,0.0,0.0,0.0,0.079621,0.000000,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10788,2024-01-29,Turquia Tb2L,12:00,High Touch,Nazilli,2.97,1.36,151.5,1.86,1.80,5.5,1.98,1.74,EkcaVga1,0.336700,0.735294,0.537634,0.555556,0.071994,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,102.60,107.30,0.0,0.0,0.0,0.0,0.525839,0.023184,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10789,2024-01-29,Turquia Tb2L,12:00,Istanbul Basket,Konya BBSK,3.86,1.23,130.5,1.83,1.83,8.5,1.96,1.75,WQ7fWZqe,0.259067,0.813008,0.546448,0.546448,0.072075,0.0,0.0,NaN,NaN,172.468,89.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Cleveland Cavaliers,Los Angeles Clippers,223.5,1.80,1.0000,Over
1,21:30,Eua Nba,Boston Celtics,New Orleans Pelicans,231.5,1.89,1.0000,Over
2,21:30,Eua Nba,Brooklyn Nets,Utah Jazz,232.5,1.89,1.0000,Over
3,22:00,Eua Nba,Memphis Grizzlies,Sacramento Kings,224.5,1.81,0.9999,Over
4,22:00,Eua Nba,Oklahoma City Thunder,Minnesota Timberwolves,223.5,1.83,0.9999,Over
5,22:00,Eua Nba,San Antonio Spurs,Washington Wizards,240.5,1.84,0.9962,Over
6,23:00,Eua Nba,Denver Nuggets,Milwaukee Bucks,238.5,1.88,1.0000,Over
7,21:30,Eua Ncaa,Delaware State,N.Carolina Central,133.5,1.91,0.8546,Over
